In [2]:
# Initial setup
import os
import pickle
import librosa
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from google.colab import drive
from collections import Counter
np.random.seed(1234)

drive.mount('/content/drive')
%cd "drive/My Drive/ptsa"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/ptsa


# Extract file labels
###  File naming convention

Each of the 7356 RAVDESS files has a unique filename. The filename consists of a 7-part numerical identifier (e.g., 02-01-06-01-02-01-12.mp4). These identifiers define the stimulus characteristics:

- Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
- Vocal channel (01 = speech, 02 = song).
- Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
- Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the ‘neutral’ emotion.
- Statement (01 = “Kids are talking by the door”, 02 = “Dogs are sitting by the door”).
- Repetition (01 = 1st repetition, 02 = 2nd repetition).
- Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

### Filename example
02-01-06-01-02-01-12.mp4 :
Video-only (02)
Speech (01)
Fearful (06)
Normal intensity (01)
Statement “dogs” (02)
1st Repetition (01)
12th Actor (12)
Female, as the actor ID number is even.

In [0]:
# Extract file names
path = "/content/drive/My Drive/ptsa/raw_data"
file_name_list = []
for subdir, dirs, files in os.walk(path):
    for file_name in files:
        if "wav" in file_name:
          file_name_list.append(file_name)

### Important labels
- Emotion
- Emotion intensity
- Statement
- Repetition
- Actor

In [0]:
# Extract labels
emotion = []
intensity = []
statement = []
repetition = []
actor = []
for name in file_name_list:
    label = name.split("-")
    emotion.append(int(label[2]))
    intensity.append(int(label[3]))
    statement.append(int(label[4]))
    repetition.append(int(label[5]))
    actor.append(int(label[6].split(".")[0]))

# Create a meta file
meta_df = pd.DataFrame({"file_name":file_name_list,
                        "emotion":emotion,
                        "intensity":intensity,
                        "statement":statement,
                        "repetition":repetition,
                        "actor":actor})

# Emotion dictionary
emotion_dict = dict({1:"neutral", 2:"calm", 3:"happy", 4:"sad", 5:"angry", 6:"fearful", 7:"disgust", 8:"surprise"})
# Add emotion label (str)
meta_df["emotion_str"] = [emotion_dict[e] for e in meta_df["emotion"]]
# Substract 1 from the current emotion label (num)
meta_df["emotion"] = meta_df["emotion"] - 1

In [6]:
meta_df.head()

,file_name,emotion,intensity,statement,repetition,actor,emotion_str
0,03-01-01-01-01-02-06.wav,0,1,1,2,6,neutral
1,03-01-01-01-02-02-06.wav,0,1,2,2,6,neutral
2,03-01-01-01-02-01-06.wav,0,1,2,1,6,neutral
3,03-01-01-01-01-01-06.wav,0,1,1,1,6,neutral
4,03-01-02-01-01-01-06.wav,1,1,1,1,6,calm


# Basic stat

In [0]:
# Class distribution
meta_df.groupby("emotion_str")["file_name"].count()

emotion_str
angry       192
calm        192
disgust     192
fearful     192
happy       192
neutral      96
sad         192
surprise    192
Name: file_name, dtype: int64

In [0]:
# Number of files per actor
meta_df.groupby("actor")["file_name"].count()

actor
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
17    60
18    60
19    60
20    60
21    60
22    60
23    60
24    60
Name: file_name, dtype: int64

# Split into train and validation

In [0]:
from sklearn.model_selection import train_test_split
# Stratified sampling
train, _ = train_test_split(meta_df["file_name"], train_size=0.75, stratify=meta_df["emotion"])
meta_df["train"] = [file_name in list(train) for file_name in meta_df["file_name"]]

In [25]:
meta_df.groupby(["emotion_str", "train"])["file_name"].count()

emotion_str  train
angry        False     48
             True     144
calm         False     48
             True     144
disgust      False     48
             True     144
fearful      False     48
             True     144
happy        False     48
             True     144
neutral      False     24
             True      72
sad          False     48
             True     144
surprise     False     48
             True     144
Name: file_name, dtype: int64

In [0]:
# Save meta data
meta_df.to_csv("meta_data.csv", index=False)